## 05 컨텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

### 장르 속성을 이용한 영화 컨텐츠 기반 필터링

In [18]:
# 컨텐츠 기반 필터링은 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성 요소 등을 가진
# 다른 영화를 추천하는 것이다.
# 영화(상품/서비스) 간의 유사성을 판단하는 기분이 영화를 구성하는 다양한 컨텐츠(장르, 감독, 배우, 평점, 키워드, 영화 설명)를
# 기반으로 하는 방식이 바로 컨텐츠 기반 필터링이다.

### 데이터 로딩 및 가공 

In [19]:
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('/Users/adam/Data_Analytics/Python/Datasets/movie_data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [20]:
# 4803개의 레코드와 20개의 피처로 구성되어 있다.
# 영화 제목, 개요, 인기도, 평점, 투표 수, 예산, 키워드 등 영화에 대한 다양한 메타 정보를 가지고 있다.
# 이 중 컨텐츠 기반 필터링 추천 분석에 사용할 주요 컬럼만 추출해 새롭게 DataFrame을 만든다.
# 추출할 주요 컬럼은 id, 영화제목 title, 영화가 속한 장르 genres, 평점 평균 vote_average, 평균 점수 투표 수 vote_count,
# 영화의 인기도 popularity, 영화 설명 주요 키워드 문구 keywords, 영화 개요 설명 overview 이다.

In [21]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count','popularity', 'keywords','overview']]

In [22]:
# 이 파일을 DataFrame 처리할 때 주의해야 할 컬럼이 있다.
# 'genres', 'keywords' 등과 같은 컬럼을 보면 파이썬 리스트(list) 내부에 여러 개의 딕셔너리(dict)가 있는 형태의 문자열로 표기 되어있다.
# 이는 한꺼번에 여러 개의 값을 표현 하기 위한 표기 방식이다.
# 이 컬럼이 DataFrame으로 만들어질 때는 단순히 문자열 형태로 로딩되므로 이 컬럼을 가공하지 않고는 필요한 정보를 추출할 수 없다.

In [23]:
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [24]:
# genre 컬럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출하겠다.
# 파이썬 ast 모듈의 literal_eval() 함수를 이용하면 이 문자열을 문자열이 의미하는 list [dict1, dict2] 객체로 만들 수 있다.

In [25]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [26]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 컨텐츠 유사도 측정 

In [27]:
# genres 컬럼은 여러 개의 개별 장르가 리스트로 구성되어 있다. 여러 개 장르별 유사도 측정은
# 가장 간단한 방법으로 genres를 문자열로 변경한 뒤에 이를 CountVectorizer로 피처 벡터화 한 행렬 데이터 값을
# 코사인 유사도로 비교하는 것이다.

# genres 컬럼을 기반으로 하는 컨텐츠 기반 필터링은 다음 단계로 구현한다.
# 1. 문자열로 변환된 genres 컬럼을 Count 기반으로 피처 벡터화 변환한다.
# 2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교한다.
#    이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성한다.
# 3. 장르 유사도가 노은 영화 중에 평점이 높은 순으로 영화를 추천한다.

In [28]:
# 리스트 객체 내의 개별 값을 연속된 문자열로 변환하려면 일반적으로 ('구분문자').join(리스트 객체)를 사용하면 된다.

from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [29]:
# CountVectorizer로 변환해 4803개의 레코드와 276개의 개별 단어 피처로 구성된 피처 벡터 행렬이 만들어 졌다.

In [31]:
# 이제 코사인 유사도를 계산해보자.

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [32]:
# argsort()[:, ::-1] 을 이용하면 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값을 간편하게 얻을 수 있다.

genre_sim_sorted_ind = genre_si.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


In [33]:
# 반환 값이 의미하는 것은 0번 레코드의 경우 자신인 0번 레코드를 제외하면 3494번 레코드가 가장 유사도가 높고,
# 그 다음이 813번 레코드이며, 가장 유사도가 낮은 레코드는 2401번 레코드 라는 뜻이다.

### 장르 컨텐츠 필터링을 이용한 영화 추천 

In [34]:
# 장르 유사도에 따라 영화를 추천하는 함수를 생성하자.

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movie_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_si_sortd_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터임.
    # dataFrmae에서 index로 사용하기 위해서 1차원 array로 변경.
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [36]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [37]:
# 이번에는 일단 좀 더 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링을 최종 추천하는 방식으로 변경해보자.
# vote_average는 0 부터 10점 만점 까지의 점수로 되어 있는데, 여러 관객이 평가한 평점을 평균한 것이다.
#  그런데 1명, 2명의 소수의 관객이 특정 영화에 만점이나 매우 높은 평점을 부여해 왜곡된 데이터를 가지고 있다.

movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [38]:
# 이름도 들어본 적 없는 영화가 높은 평점을 받고 있는데, 이들 모두 평가 횟수가 매우 작다.
# 이와 같은 왜곡된 평점 데이터를 회피할 수 있도록 평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요하다.

# 유명한 영화 평점 사이트인 IMDB에서는 평가 횟수에 대한 가중치가 부여된 평점(Weighted Rating)방식을 사용한다.

# 가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C

# v: 개별 영화에 평점을 투표한 횟수
# m: 평점을 부여하기 위한 최소 투표 횟수
# R: 개별 영화에 대한 평균 평점.
# C: 전체 영화에 대한 평균 평점.

# m 값은 전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정한다.
# 상위 60% 값은 Series 객체의 quantile()을 이용해 추출한다.

In [40]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


In [41]:
# 기존 평점을 새로운 가중 평점으로 변경하는 함수를 생성하고 이를 이용해 새로운 평점 정보인 'vote_weighted' 값을 만들어 보자.

percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m)) * R) + ((m/(m+v)) * C)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [42]:
# 새롭게 부여된 weighted_vote 평점이 높은 순으로 상위 10개의 영화 순위를 추출해보자.

movies_df[['title','vote_average', 'weighted_vote','vote_count']].sort_values('weighted_vote',
                                                                             ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [43]:
# TOP 10에 대한 개인별 성향이 조금씩 달라서 위 결과에 이의가 있을지는 몰라도 위 영화 모두 매우 뛰어난 영화라는 점에는 이견이 없을 것이다.

In [44]:
# 이제 새롭게 정의된 평점 기준에 따라 영화를 추천해보자.

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 장르 유사성이 높은 index cncnf
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
